# Advanced Compilation Techniques

<p>This notebook demonstrates selected advanced language features and compilation techniques, including</p>
1. Parfor: Parallel for loops for task-parallel computation (language, compiler, and runtime)
2. Codegen: Code generation for operator fusion (compiler/runtime)

In [ ]:
from systemml import MLContext, dml, dmlFromResource, jvm_stdout

ml = MLContext(sc)
ml.setStatistics(True)

print "Spark Version:", sc.version
print "SystemML Version:", ml.version()
print "SystemML Built-Time:", ml.buildTime()

## 1 Parfor: Parallel For Loops for Task-Parallel Computation 

<p>Parfor loops allow for arbitrary task-parallel computation, which requires independence of iterations, i.e., the absence of loop-carried dependencies on result variables.</p>

### A Simple Example of Univariate Descriptive Statistics

<p>Compute column-wise mean, standard deviation, skewnes, and kurtosis.</p> 

<p>a) Small data scenario (local parfor)</p>

In [ ]:
prog = """
X = rand(rows=10000, cols=100);
R = matrix(0, ncol(X), 4)
parfor(i in 1:ncol(X)) {
   Xi = X[,i];
   R[i,1] = mean(Xi);                  # mean
   R[i,2] = sqrt(var(Xi));             # standard deviation
   R[i,3] = moment(Xi,3)/(R[i,2]^3);   # skewness
   R[i,4] = moment(Xi,4)/(R[i,2]^4)-3; # kurtosis
}
"""
with jvm_stdout():
    R = ml.execute(dml(prog).output("R")).get("R")
R.toNumPy()[1:4]

<p>b) ParFor EXPLAIN: Understanding task-parallel execution plans and optimizer choices.</p> 

In [ ]:
prog = """
X = rand(rows=10000, cols=100);
R = matrix(0, ncol(X), 4)
parfor(i in 1:ncol(X), log=DEBUG) {
   Xi = X[,i];
   R[i,1] = mean(Xi);                  # mean
   R[i,2] = sqrt(var(Xi));             # standard deviation
   R[i,3] = moment(Xi,3)/(R[i,2]^3);   # skewness
   R[i,4] = moment(Xi,4)/(R[i,2]^4)-3; # kurtosis
}
"""
with jvm_stdout():
    R = ml.execute(dml(prog).output("R")).get("R")
R.toNumPy()[1:4]

<p>c) Medium-sized data scenario (remote_spark parfor w/ and w/o fused data partitioning)</p>

In [ ]:
prog = """
X = rand(rows=100000, cols=100);
R = matrix(0, ncol(X), 4)
parfor(i in 1:ncol(X), log=DEBUG) {
   Xi = X[,i];
   R[i,1] = mean(Xi);                  # mean
   R[i,2] = sqrt(var(Xi));             # standard deviation
   R[i,3] = moment(Xi,3)/(R[i,2]^3);   # skewness
   R[i,4] = moment(Xi,4)/(R[i,2]^4)-3; # kurtosis
}
"""

with jvm_stdout():
    R = ml.execute(dml(prog).output("R")).get("R")
R.toNumPy()[1:4]

### An Example of Bivariate Descriptive Statistics w/ Nested Parallelism
<p>Compute the chi-square test statistic for all pairs of categorical columns.</p> 

In [ ]:
prog = """
D = 20;
X = rand(rows=10000, cols=100, min=D, max=D+1);
R = matrix(0, ncol(X), ncol(X));

parfor(i in 1:(ncol(X)-1), log=DEBUG) {
  parfor(j in (i+1):ncol(X)) {
    # Chi-Squared
    F = table(X[,i], X[,j], D, D);
    W = sum(F);
    r = rowSums(F);
    c = colSums(F);
    E = (r %*% c) / W;
    T = (F-E)^2 / E;
    R[i,j] = sum(T);
  }
}
"""

with jvm_stdout():
    R = ml.execute(dml(prog).output("R")).get("R")
R.toNumPy()[1:4]

## 2 Codegen: Code Generation for operator fusion

<p>SystemML's code generator identifies and optimizes fusion plans and generates efficient java source code for these fused operators via Janino.</p>
<p>Fusion opportunities are ubiquitous and help to avoid materialized intermediates, reduce the number of scans, and exploit sparsity across chains of operations.</p>

In [ ]:
ml.setConfigProperty("optlevel", "1")           #disable rewrites (demo only)
ml.setConfigProperty("codegen.enabled", "true") #enable codegen

### Simple Example Expressions using Different Templates  

<p>a) Cell-wise templates w/ aggregation</p>

In [ ]:
prog = """
X = rand(rows=10000, cols=1);
Y = rand(rows=10000, cols=1)
Z = rand(rows=10000, cols=1)
R = sum(X * Y * Z)
"""

ml.setExplain(True)
with jvm_stdout():
    R = ml.execute(dml(prog).output("R")).get("R")
R

<p>b) Row-wise template w/ aggregation</p>

In [ ]:
prog = """
X = rand(rows=10000, cols=10);
v = rand(rows=10, cols=3);
P = rand(rows=10000, cols=4);
K = 3;

Q = P[,1:K] * (X %*% v);
R = t(X) %*% (Q - P[,1:K] * rowSums(Q));
"""

ml.setExplain(True)
with jvm_stdout():
    R = ml.execute(dml(prog).output("R")).get("R")
R

<p>c) Multi-aggregate template</p>

In [ ]:
prog = """
X = rand(rows=10000, cols=1);
Y = rand(rows=10000, cols=1);

R1 = sum(X^2);
R2 = sum(X*Y);
R3 = sum(Y^2);
"""

ml.setExplain(True)
with jvm_stdout():
    R1 = ml.execute(dml(prog).output("R1","R2","R3")).get("R1")
R1

<p>d) Outer-product template</p>

In [ ]:
prog = """
W = rand(rows=10000, cols=10000, sparsity=0.1);
S = rand(rows=10000, cols=10);
V = rand(rows=10000, cols=10);
RNZ = rand(rows=10000, cols=1);

R = (W * (S %*% t(V))) %*% V + 1e-6 * S * RNZ;
"""

ml.setExplain(True)
with jvm_stdout():
    R = ml.execute(dml(prog).output("R")).get("R")
R.toNumPy()[1:4]

### An End-to-End Algorithm: L2SVM

<p>L2SVM without codegen:</p>

In [ ]:
#ml.setConfigProperty("optlevel", "2")            #disable rewrites (demo only)
#ml.setConfigProperty("codegen.enabled", "false") #enable codegen

ml.setStatistics(True)

dmlscript = dmlFromResource("/scripts/algorithms/l2-svm.dml").input(
'$X','data/higgs_features').input('$Y','data/higgs_labels').input(
'$Log', 'log/out.log').input('icpt',"0").input('$tol','1e-14').input(
'$reg','1e-3').input('$maxiter','50').output("w")

with jvm_stdout():
    beta = ml.execute(dmlscript).get("w")
beta.toNumPy()

<p>L2SVM with codegen:</p>

In [ ]:
#ml.setConfigProperty("optlevel", "1")            #disable rewrites (demo only)
#ml.setConfigProperty("codegen.enabled", "true") #enable codegen

ml.setStatistics(True)

dmlscript = dmlFromResource("/scripts/algorithms/l2-svm.dml").input(
'$X','data/higgs_features').input('$Y','data/higgs_labels').input(
'$Log', 'log/out.log').input('icpt',"0").input('$tol','1e-14').input(
'$reg','1e-3').input('$maxiter','50').output("w")

with jvm_stdout():
    beta = ml.execute(dmlscript).get("w")
beta.toNumPy()